In [1]:
import pandas as pd
import glob
import deepl
from dotenv import load_dotenv
import os
load_dotenv() 

# df = pd.read_excel('data/nlmyo/processed/diag.xlsx')
# # Read the text content from the filename column and add it to a new column
# df["filepath"] = df["filename"].apply(lambda x: "data/nlmyo/processed/" + x)
# df['raw_text'] = df['filepath'].apply(lambda x: open(x, 'r').read())


# translator = deepl.Translator(os.getenv("DEEPL_KEY")) 
# df['deepl_translation'] = df['raw_text'].apply(lambda x: translator.translate_text(x, target_lang="EN-US").text)
# save df to csv file
# df.to_csv('data/nlmyo/processed/diag_translated.csv', index=False)

True

In [2]:
import numpy as np
from embetter.external import CohereEncoder, OpenAIEncoder
from embetter.grab import ColumnGrabber
from sklearn.pipeline import make_pipeline 
import pandas as pd
from dotenv import load_dotenv
import os

from cohere import Client

# load_dotenv() 
# client = Client(os.getenv("COHERE_KEY"))
# # This pipeline grabs the `text` column from a dataframe

# df = pd.read_csv('data/nlmyo/processed/diag_translated.csv')

# text_emb_pipeline = make_pipeline(
#   ColumnGrabber("deepl_translation"),
#   CohereEncoder(client=client, model="large")
# )

# X = text_emb_pipeline.fit_transform(df, df['diag_simple'])
# np.save('data/nlmyo/processed/report_translated_embed_cohere.npy', X)

In [3]:
import numpy as np
from embetter.external import CohereEncoder, OpenAIEncoder
from embetter.grab import ColumnGrabber
from sklearn.pipeline import make_pipeline 
import pandas as pd
from dotenv import load_dotenv
import os

import openai

# load_dotenv() 

# openai.organization = os.getenv("OPENAI_ORG")
# openai.api_key = os.getenv("OPENAI_KEY")
# # This pipeline grabs the `text` column from a dataframe

# df = pd.read_csv('data/nlmyo/processed/diag_translated.csv')

# text_emb_pipeline = make_pipeline(
#   ColumnGrabber("deepl_translation"),
#   OpenAIEncoder(model="text-embedding-ada-002")
# )

# X = text_emb_pipeline.fit_transform(df, df['diag_simple'])
# np.save('data/nlmyo/processed/report_translated_embed_openai.npy', X)

In [6]:
df = pd.read_csv('data/nlmyo/processed/diag_translated.csv')
df['diag_simple'].value_counts()

Nermaline Myopathy        37
Core Myopathy             37
UNCLEAR                   37
Centronuclear Myopathy    12
NON-MC                    11
CFTD                       2
Name: diag_simple, dtype: int64

In [2]:
X_cohere.shape

(97, 4096)

In [7]:
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier

#### Import the data
df = pd.read_csv('data/nlmyo/processed/diag_translated.csv')
Y = df['diag_simple'].values
X_cohere = np.load('data/nlmyo/processed/report_translated_embed_cohere.npy') 
X_openai = np.load('data/nlmyo/processed/report_translated_embed_openai.npy')

# Remove CFTD and unclear diagnosis
df['diag_simple'].value_counts()
df['diag_simple'] = df['diag_simple'].replace('CFTD', 'UNCLEAR')
# Drop the rows with unclear diagnosis
df = df[df['diag_simple'] != 'UNCLEAR']
Y = df['diag_simple'].values
# Do the same for the X array based on the df index
X_cohere = X_cohere[df.index]
X_openai = X_openai[df.index]
cv_fold = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

In [5]:
clf_dummy = DummyClassifier(strategy='prior')
cv_scores_dummy = cross_val_score(clf_dummy, X_cohere, Y, cv=cv_fold)
print("Dummy Classifier Results:")
print(f"All CV Scores: {cv_scores_dummy}")
print(f"Mean CV Score:  {np.mean(cv_scores_dummy)}")
print(f"Standard Deviation CV Score: {np.std(cv_scores_dummy)}")

Dummy Classifier Results:
All CV Scores: [0.4        0.4        0.4        0.4        0.3        0.3
 0.3        0.33333333 0.33333333 0.33333333]
Mean CV Score:  0.35000000000000003
Standard Deviation CV Score: 0.04281744192888378


In [6]:
clf = LogisticRegression(max_iter=3000)
cv_scores = cross_val_score(clf, X_cohere, Y, cv=cv_fold)
print("Results with Logistic Regression and Cohere Embeddings on English Translated Reports:")
print(f"All CV Scores: {cv_scores}")
print(f"Mean CV Score:  {np.mean(cv_scores)}")
print(f"Standard Deviation CV Score: {np.std(cv_scores)}")

Results with Logistic Regression and Cohere Embeddings on English Translated Reports:
All CV Scores: [0.6        0.7        0.6        0.5        0.6        0.5
 0.7        0.55555556 0.66666667 0.77777778]
Mean CV Score:  0.62
Standard Deviation CV Score: 0.08603760303380416


In [7]:
clf = LogisticRegression(max_iter=3000)
cv_scores = cross_val_score(clf, X_openai, Y, cv=cv_fold)
print("Results with Logistic Regression and OpenAI Embeddings on English Translated Reports:")
print(f"All CV Scores: {cv_scores}")
print(f"Mean CV Score:  {np.mean(cv_scores)}")
print(f"Standard Deviation CV Score: {np.std(cv_scores)}")

Results with Logistic Regression and OpenAI Embeddings on English Translated Reports:
All CV Scores: [0.6        0.4        0.6        0.6        0.5        0.5
 0.6        0.77777778 0.44444444 0.66666667]
Mean CV Score:  0.5688888888888889
Standard Deviation CV Score: 0.10515127257174203


In [1]:
from sklearn.base import BaseEstimator
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB

class DummyEstimator(BaseEstimator):
    def fit(self): pass
    def score(self): pass


# Create a pipeline
pipe = Pipeline([('clf', DummyEstimator())]) # Placeholder Estimator

# Candidate learning algorithms and their hyperparameters
search_space = [{'clf': [LogisticRegression()],
                    'clf__max_iter': [1500]},
                {'clf': [GaussianNB()],},
                {'clf': [MLPClassifier()],
                 'clf__max_iter': [500]},
                {'clf': [KNeighborsClassifier()],},
                {'clf': [SVC()],},
                {'clf': [GaussianProcessClassifier()],},
                {'clf': [HistGradientBoostingClassifier()],},
                {'clf': [DecisionTreeClassifier()],},
                {'clf': [RandomForestClassifier()],},
                {'clf': [AdaBoostClassifier()],},
                ]


In [ ]:
# Create grid search 
gs = GridSearchCV(pipe, search_space, scoring="accuracy", cv=cv_fold)
gs.fit(X_cohere, Y)
df_cv_search = pd.DataFrame(gs.cv_results_)
df_cv_search.to_csv('data/nlmyo/processed/report_translated_embed_cohere_gridsearch.csv')
df_cv_search

In [9]:
# Create grid search 
gs = GridSearchCV(pipe, search_space, scoring="accuracy", cv=cv_fold)
gs.fit(X_openai, Y)
df_cv_search = pd.DataFrame(gs.cv_results_)
df_cv_search.to_csv('data/nlmyo/processed/report_translated_embed_cohere_gridsearch.csv')
df_cv_search

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_clf,param_clf__max_iter,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,mean_test_score,std_test_score,rank_test_score
0,0.038400,0.004094,0.001046,0.000041,LogisticRegression(),1500,"{'clf': LogisticRegression(), 'clf__max_iter':...",0.6,0.4,0.6,0.6,0.5,0.5,0.6,0.777778,0.444444,0.666667,0.568889,0.105151,4
1,0.002641,0.000271,0.001127,0.000062,GaussianNB(),NaN,{'clf': GaussianNB()},0.4,0.4,0.4,0.4,0.3,0.4,0.6,0.888889,0.333333,0.444444,0.456667,0.162272,9
2,2.746184,0.364329,0.001606,0.000231,MLPClassifier(max_iter=1000),1000,"{'clf': MLPClassifier(max_iter=1000), 'clf__ma...",0.6,0.7,0.7,0.7,0.8,0.2,0.8,0.777778,0.555556,0.888889,0.672222,0.182996,1
3,0.001071,0.000153,0.013386,0.016027,KNeighborsClassifier(),NaN,{'clf': KNeighborsClassifier()},0.6,0.6,0.3,0.5,0.5,0.3,0.3,0.888889,0.444444,0.444444,0.487778,0.172021,7
4,0.005151,0.001714,0.001672,0.000073,SVC(),NaN,{'clf': SVC()},0.5,0.4,0.5,0.4,0.4,0.4,0.7,0.666667,0.444444,0.555556,0.496667,0.106696,6
5,0.084241,0.014789,0.011682,0.003856,GaussianProcessClassifier(),NaN,{'clf': GaussianProcessClassifier()},0.6,0.4,0.6,0.6,0.5,0.4,0.6,0.666667,0.444444,0.666667,0.547778,0.097885,5
6,1.456280,0.087878,0.013010,0.005593,HistGradientBoostingClassifier(),NaN,{'clf': HistGradientBoostingClassifier()},0.6,0.6,0.5,0.4,0.7,0.2,0.6,0.777778,0.555556,0.777778,0.571111,0.166385,3
7,0.050843,0.013542,0.000726,0.000181,DecisionTreeClassifier(),NaN,{'clf': DecisionTreeClassifier()},0.5,0.3,0.5,0.7,0.2,0.3,0.4,0.555556,0.333333,0.444444,0.423333,0.139704,10
8,0.267129,0.006483,0.014863,0.000295,RandomForestClassifier(),NaN,{'clf': RandomForestClassifier()},0.5,0.6,0.5,0.7,0.7,0.4,0.6,0.888889,0.666667,0.777778,0.633333,0.137257,2
9,0.813311,0.074244,0.010220,0.000613,AdaBoostClassifier(),NaN,{'clf': AdaBoostClassifier()},0.5,0.4,0.6,0.5,0.5,0.4,0.4,0.555556,0.444444,0.333333,0.463333,0.077786,8
